In [62]:
HOME = "/data/git/shopee-product-matching"
p_out = f'{HOME}/output/prep002'
!mkdir -p {p_out}



import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

pcomp = f'{HOME}/input/shopee-product-matching'
df = pd.read_csv(f'{pcomp}/train.csv')
df['filepath'] = f"{pcomp}/train_images/"+df.image

n_grps = df.label_group.nunique()

grps = np.array(sorted(df.label_group.unique()))
np.random.seed(101)
msk_train = np.random.rand(n_grps)>0.15
grps_train = set(grps[msk_train])
grps_valid = set(grps[~msk_train])
assert grps_train & grps_valid == set() and grps_train|grps_valid==set(grps)

df_train = df[df.label_group.isin(grps_train)].copy().reset_index()
df_valid = df[df.label_group.isin(grps_valid)].copy().reset_index()
print(df.shape, df_train.shape, df_valid.shape)

(34250, 6) (29115, 7) (5135, 7)


In [63]:
df_train.head()

,index,posting_id,image,image_phash,title,label_group,filepath
0,0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,/data/git/shopee-product-matching/input/shopee...
1,1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,/data/git/shopee-product-matching/input/shopee...
2,2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,/data/git/shopee-product-matching/input/shopee...
3,3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,/data/git/shopee-product-matching/input/shopee...
4,4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,/data/git/shopee-product-matching/input/shopee...


In [64]:
df_valid.head()

,index,posting_id,image,image_phash,title,label_group,filepath
0,6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137,/data/git/shopee-product-matching/input/shopee...
1,7,train_1806152124,0014f61389cbaa687a58e38a97b6383d.jpg,eea7e1c0c04da33d,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT...,1565741687,/data/git/shopee-product-matching/input/shopee...
2,16,train_3009013664,0027aaf8dd8bdbf0e4f2c19024e436cf.jpg,b3cccc26cc3333cc,MARKS & SPENCER - Rose Hand & Body Lotion 250 ml,1574620312,/data/git/shopee-product-matching/input/shopee...
3,18,train_2985955659,002f978c58a44a00aadfca71c3cad2bb.jpg,bf38f0e083d7c710,HnKfashion Sweater Hoodie WHO Printing BabyTer...,3415582503,/data/git/shopee-product-matching/input/shopee...
4,21,train_2238403912,003524b70715bf6bfa00451ca08e66e0.jpg,ba35c44a3fb7c068,Kangaroo Teflon / Allu Fry Pan 18 cm - KG652,531768918,/data/git/shopee-product-matching/input/shopee...


In [65]:
skf = StratifiedKFold(5, shuffle=True, random_state=101)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train.label_group)):
    df_train.loc[valid_idx, 'fold'] = i

df_train.to_csv(f'{p_out}/df_train.csv', index=False)
df_valid.to_csv(f'{p_out}/df_valid.csv', index=False)

/data/anaconda3/envs/shopee/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
